In [1]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.chat_models.ollama import ChatOllama
from langchain_core.callbacks import StreamingStdOutCallbackHandler
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationSummaryBufferMemory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# 챗 지피티
chat_ollama = ChatOpenAI(
    temperature=0.1,
    model="gpt-3.5-turbo",
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# 로컬 LLM (ollama)
chat = ChatOllama(
    temperature=0.1,
    model="openhermes:latest",
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


example = [
    {
        "영화제목": "탑건",
        "answer": "🛩️👨‍✈️🔥",
    },
    {
        "영화제목": "대부",
        "answer": "👨‍👨‍👦🔫🍝",
    },
    {
        "영화제목": "쥬라기 공원",
        "answer": "🦖🏞️🚗",
    },
    {
        "영화제목": "해리 포터",
        "answer": "🧙‍♂️⚡🏰",
    },
    {
        "영화제목": "토이 스토리",
        "answer": "🤠🚀🧸",
    },
    {
        "영화제목": "타이타닉",
        "answer": "🚢💏🧊",
    },
    {
        "영화제목": "인셉션",
        "answer": "💤🌀🕰️",
    },
    {
        "영화제목": "어벤져스",
        "answer": "🦸‍♂️🦸‍♀️🌌",
    },
    {
        "영화제목": "라이온 킹",
        "answer": "🦁👑🌅",
    },
    {
        "영화제목": "겨울왕국",
        "answer": "❄️👸🎶",
    },
    {
        "영화제목": "매트릭스",
        "answer": "🕶️💊🐇",
    },
    {
        "영화제목": "스타워즈",
        "answer": "⭐🚀🤖",
    },
]


memory = ConversationSummaryBufferMemory(
    llm=chat,
    max_token_limit=120,
    memory_key="history",
    return_messages=True,
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


output_parser = StrOutputParser()

example_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "human",
            "영화 제목을 말하면 영화를 설명하는 이모티콘으로 답변해줘. 영화제목은 {영화제목}야",
        ),
        ("ai", "{영화제목} : {answer}\n"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=example,
)


explain_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 영화에 대해 이모티콘 3개로 설명할 수 있어."),
        example_prompt,
        ("human", "{영화제목}"),
    ]
)

find_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 이제까지의 대화내용을 나에게 말해 줄 수 있어."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


# 체인 생성 (중간에 chat_ollama를 넣어서 로컬 LLM을 사용함 -> chat으로 바꾸면 오픈AI 사용)
explain_chain = explain_prompt | chat | output_parser

find_chain = (
    RunnablePassthrough.assign(history=load_memory) | find_prompt | chat | output_parser
)


def invoke_chain(question):
    result = explain_chain.invoke({"영화제목": question})
    memory.save_context(
        {"input": question},
        {"output": result},
    )
    print("")

In [2]:
invoke_chain("로보캅")

invoke_chain("석양의 무법자")

invoke_chain("핑크팬더")

로보캅 : 🤖🐎🔥

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


SSLError: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /gpt2/resolve/main/tokenizer_config.json (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1006)')))"), '(Request ID: b0722730-3068-4f93-abf1-57475d26ef72)')

In [3]:
find_chain.invoke({"question": "내가 첫번째에 물어본 영화제목은 뭐야?"})
print("")
find_chain.invoke({"question": "내가 두번째에 물어본 영화제목은 뭐야?"})
print("")
find_chain.invoke({"question": "내가 세번째에 물어본 영화제목은 뭐야?"})
print("")
memory.load_memory_variables({})["history"]

네가 첫 번째로 물어본 영화 제목은 "로보캅"이었어.
두번째로 물어본 영화 제목은 "로보캅"이야.
세번째로 물어본 영화 제목은 "로보캅"이야.


[HumanMessage(content='로보캅'), AIMessage(content='로보캅: 🤖👮\u200d♂️🔫')]